In [1]:
import pandas as pd
import os
import numpy as np
import heapq
import editdistance as Levdist
from pyxdameraulevenshtein import damerau_levenshtein_distance as DLevdist
from pyxdameraulevenshtein import normalized_damerau_levenshtein_distance as NDLevdist

In [2]:
# actdats - start date
# cname - company name
# analyst - analyst
# baname - analyst
# estimid - bank

In [3]:
recdet = pd.read_stata('../data/recdet.dta')
bran = pd.read_stata('../data/bran.dta')

In [4]:
merged = pd.merge(recdet,
                 bran,
                 how='outer',
                 left_on=['amaskcd'],
                 right_on=['bacode'])

In [5]:
merged.head()

,ticker,cusip,cname,oftic,actdats,estimid,analyst,ereccd,etext,ireccd,...,acttims,revdats,revtims,anndats,anntims,baindi,bacode,baname,baid,usfirm_y
0,0000,87482X10,TALMER BANCORP,TLMR,2014-03-10,RBCDOMIN,ARFSTROM J,2,OUTPERFORM,2,...,08:54:03,2016-01-26,09:35:52,2014-03-10,00:20:00,A,71182.0,ARFSTROM J,,1.0
1,0000,87482X10,TALMER BANCORP,TLMR,2016-01-26,RBCDOMIN,ARFSTROM J,3,SECTOR PERFORM,3,...,21:57:09,2016-04-28,10:48:13,2016-01-26,21:44:00,A,71182.0,ARFSTROM J,,1.0
2,0097,39260X10,GREEN BANCORP,GNBC,2014-09-02,RBCDOMIN,ARFSTROM J,2,OUTPERFORM,2,...,08:13:22,2016-07-28,21:40:22,2014-09-02,07:50:00,A,71182.0,ARFSTROM J,,1.0
3,AINV,03761U10,APOLLO INVESTMEN,AINV,2009-08-08,RBCDOMIN,ARFSTROM J,2,OUTPERFORM,2,...,18:58:54,2017-05-18,09:22:18,2009-08-07,06:53:00,A,71182.0,ARFSTROM J,,1.0
4,ALLE,01747610,ALLEGIANT BANC,ALLE,2002-01-14,RBCDOMIN,ARFSTROM J,2,OUTPERFORM,2,...,16:22:51,2003-10-27,15:08:44,2002-01-14,16:16:21,A,71182.0,ARFSTROM J,,1.0


In [6]:
for c in merged.columns:
    print(c, ' -- ', merged[c][0], ' -- ', merged[c][1])

ticker  --  0000  --  0000
cusip  --  87482X10  --  87482X10
cname  --  TALMER BANCORP  --  TALMER BANCORP
oftic  --  TLMR  --  TLMR
actdats  --  2014-03-10 00:00:00  --  2016-01-26 00:00:00
estimid  --  RBCDOMIN  --  RBCDOMIN
analyst  --  ARFSTROM      J  --  ARFSTROM      J
ereccd  --  2  --  3
etext  --  OUTPERFORM  --  SECTOR PERFORM
ireccd  --  2  --  3
itext  --  BUY  --  HOLD
emaskcd  --  659.0  --  659.0
amaskcd  --  71182.0  --  71182.0
usfirm_x  --  1.0  --  1.0
acttims  --  08:54:03  --  21:57:09
revdats  --  2016-01-26 00:00:00  --  2016-04-28 00:00:00
revtims  --  09:35:52  --  10:48:13
anndats  --  2014-03-10 00:00:00  --  2016-01-26 00:00:00
anntims  --  00:20:00  --  21:44:00
baindi  --  A  --  A
bacode  --  71182.0  --  71182.0
baname  --  ARFSTROM           J                      --  ARFSTROM           J                    
baid  --    --  
usfirm_y  --  1.0  --  1.0


In [41]:
def sort_by_levenstein(list1, list2):
    if len(list1)-len(list2)>=0:
        listb = list1
        listm = list2
        cols = ['list1', 'list2_1', 'list2_2']
    else:
        listb = list2
        listm = list1
        cols = ['list2', 'list1_1', 'list1_2']
    
    result = []
    for l1 in range(len(listb)):
        if len(listm)!=1:
            cur_lev_dist = []
            for l2 in range(len(listm)):
                cur_lev_dist.append(DLevdist(listb[l1], listm[l2]))

            min_indexes = heapq.nsmallest(3, range(len(cur_lev_dist)), cur_lev_dist.__getitem__)
            result.append([listb[l1], listm[min_indexes[0]], listm[min_indexes[1]]])
            listm.pop(min_indexes[0])
        else:
            result.append([listb[l1], np.nan, np.nan])
    return pd.DataFrame(result, columns=cols)

In [39]:
recdet_banks = pd.unique(merged['estimid'].dropna().apply(lambda x: x.replace(' ', '').lower())).tolist()

In [40]:
sa_banks = []
for f in os.listdir('../data/res_tables'):
    table = pd.read_csv('../data/res_tables/' + f)
    sa_banks += pd.unique(table['Analyst_bank'].dropna().apply(lambda x: x.replace(' ', '').lower())).tolist()
sa_banks = pd.unique(sa_banks).tolist()
sa_banks.remove('no_company_found')

In [42]:
res = sort_by_levenstein(recdet_banks, sa_banks)

In [44]:
res.to_csv('banks_first_try.csv')

In [51]:
n_lines_merged_files = len(merged)//300

In [56]:
for i in range(301):
    print(i)
    merged.iloc[n_lines_merged_files * i:n_lines_merged_files * (i+1), :].to_excel('merged/part' + str(i) + '.xlsx')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
merged[['cname', 'analyst', 'baname', 'estimid']].drop_duplicates().dropna().reset_index(drop=True)

,cname,analyst,baname,estimid
0,TALMER BANCORP,ARFSTROM J,ARFSTROM J,RBCDOMIN
1,GREEN BANCORP,ARFSTROM J,ARFSTROM J,RBCDOMIN
2,APOLLO INVESTMEN,ARFSTROM J,ARFSTROM J,RBCDOMIN
3,ALLEGIANT BANC,ARFSTROM J,ARFSTROM J,RBCDOMIN
4,ASSOC BANC CP WI,ARFSTROM J,ARFSTROM J,RBCDOMIN
5,BANCORPSOUTH INC,ARFSTROM J,ARFSTROM J,RBCDOMIN
6,BOK FINANCIAL CP,ARFSTROM J,ARFSTROM J,RBCDOMIN
7,1ST BKS AMERICA,ARFSTROM J,ARFSTROM J,DAIN
8,1ST BKS AMERICA,ARFSTROM J,ARFSTROM J,RBCDOMIN
9,CATHAY GENERAL,ARFSTROM J,ARFSTROM J,RBCDOMIN


In [5]:
recdet.to_csv('recdet.csv')

In [9]:
res_tabs = os.listdir('../data/res_tables')

analutics_sa = []

sares = pd.read_csv('../data/res_tables/' + res_tabs[0])


for f in res_tabs:
    sares = pd.read_csv('../data/res_tables/' + f)
    analutics_sa += list(pd.unique(sares['Analyst']))
    # analutics_sa += sum(list(map(lambda x: x.split(', '), pd.unique(sares['Analysts_list']))), [])

analutics_sa = pd.unique(analutics_sa)

analutics_sa = analutics_sa[~pd.isnull(analutics_sa)].copy()

In [11]:
pd.DataFrame(analutics_sa).to_csv('analytics.csv')

In [2]:
res_tabs = os.listdir('../data/res_tables')
analutics_sa = []
for f in res_tabs:
    sares = pd.read_csv('../data/res_tables/' + f)
    analutics_sa += sum(list(map(lambda x: x.split(', '), pd.unique(sares['Analysts_list']))), [])

In [3]:
l = pd.Series(analutics_sa)

In [17]:
pd.DataFrame(pd.unique(l[(l.map(len)>=17) & (l.map(len)<=55)])).to_csv('analusts_bank.csv')